# Learning review

## About Multi-agent reinforcement learning (MARL)
A  single RL agent learns through interaction with the external environment. The specific process involves the agent responding to the current state of the environment based on actions guided by its strategy. In turn, the agent receives a reward value as feedback for the quality of its actions. The primary goal of RL is to learn a strategy that maximizes rewards. Unlike supervised learning, this reward is often delayed in many scenarios.
Most successful applications of RL occur in single-agent scenarios where there's no need to model or predict the actions of other agents in the environment. However, many crucial applications involve interactions among multiple agents, leading to complex challenges. Examples include multi-robot collaboration and multiplayer games, both of which fall under multi-agent scenarios. Additionally, multi-agent self-play is an effective approach within RL. Thus, extending RL from single-agent success to multi-agent environments is vital for designing intelligent systems capable of interacting with humans or other intelligent agents.
Multi-agent systems often tackle problem-solving within unpredictable and dynamic environments, making centralized control mechanisms inadequate at predicting the behavior of each individual agent in the next step. To address this, three main solutions are commonly employed:
1. Designing effective rules that constrain the behavior of multiple agents within the system to prevent conflicts and regulate their choices.
2. Utilizing communication methods that enable agents to engage in effective exchanges, avoiding conflicts and enhancing collaboration.
3. Introducing learning mechanisms that allow agents to learn and become more intelligent through action execution and interactions.


### Relationships Among Agents:
- Fully Cooperative: Agents have aligned interests, receive the same rewards, and share common objectives.
- Fully Competitive: The reward gained by one party is at the expense of another. For instance, two robots in a competition setting.
- Mixed Cooperative & Competitive: Both competition and cooperation exist. For example, in soccer robotics, the two teams are in a competitive relationship, while within    each team, there is a cooperative dynamic.
- Self-Interested: An agent's actions can alter the environment's state, which might benefit or harm other agents. However, the agent is indifferent to this and only focuses on maximizing its own interests.


## Agents’ Communication Methods:
### Fully Decentralized: 
  Each agent interacts independently with the environment and updates its strategy based on its own observations and rewards. Agents are individual entities that do not communicate or exchange information with each other.
Agents receive observations and rewards without knowledge of others' observations and actions. Each agent has its own independent policy network and trains it separately. This process is identical to single-agent reinforcement learning. After training, each agent employs its policy network for decision-making. It inputs observed states into the policy network, which outputs a probability distribution. Actions are sampled from this distribution and then executed.
Whether in training or execution, there is no communication or interaction between agents. The essence of this decentralized approach is essentially single-agent reinforcement learning, rather than genuine multi-agent reinforcement learning. The drawback of this method is that it assumes complete independence between agents and disregards the impact of agents on each other, which is unrealistic.
### Fully Centralized: 
   All agents transmit information to a central controller. The central controller has knowledge of the observations, actions, and rewards of all agents. Agents do not have their own policy networks and do not make decisions independently. Decisions are made by the central controller, and agents simply execute instructions.
There are n agents interacting with the environment, and each agent's actions can alter the environment, consequently affecting other agents. The agents are controlled by a central controller; they lack their own policy networks and cannot make decisions independently. Instead, they rely on instructions from the central controller. During training, agents report their own observations (o) and rewards (r) to the central controller. The policy networks reside in the central controller, which makes decisions based on all observations and rewards, conveying the decisions (a) to the agents. Each agent then follows the central controller's instructions to interact with the environment through actions. Training occurs within the central controller, utilizing all observations, rewards, and actions to train the policy networks.
The central controller trains n policy networks, π(a^i│o^1, ..., o^n; θ^i), for all i=1, 2, ..., n. Different networks might share the same structure but possess different parameters. Here, θ^i  represents the parameters of the i^th policy network. The policy network takes o^1,〖……,o〗^n as inputs and determines the action a^i for the i^th agent. Decisions can only be made by the central controller because the policy network utilizes observations from all agents, while an individual agent only has access to its own observation o^i, not the entire set.
The drawback of this approach lies in its speed. Since decisions can only be made once information from all agents is collected, the overall speed is determined by the slowest member. The algorithm's slow pace is a limitation, and typically, this algorithm cannot achieve real-time decision-making.
### Centralized Training with Decentralized Execution: 
   Each agent has its own policy network, and during training, there is a central controller that collects observations, actions, and rewards from all agents. The central controller assists agents in training their policy networks. After training, the central controller is no longer used. Each agent makes decisions based on its observations using its own policy network, without the need to communicate with the central controller.
The system employs n agents, each with a policy network on their respective agents (actors). Agents interact with the environment, obtaining o^i, r^i from it. Agents make decisions to generate actions a^i, which are then executed. During training, a central controller is necessary. Agents communicate with the central controller, sending actions, observations, and rewards for processing.
The central controller possesses information from all agents: {a^i,o^i,r^i }_(i=1)^n. The central controller holds n value networks, 〖Critic〗^i, where each value network corresponds to an agent. The central network trains value networks q(o,a;w^i ) using a TD algorithm to update w^i.
After training, the central controller is no longer needed. Each agent interacts independently with the environment, observing a local environment. Each agent possesses its own decision network π(∙│o^i 〖;θ〗^i ), taking observations as input and producing an action probability distribution. Actions a^i are chosen based on this distribution, and each agent executes their selected action. Subsequently, the environment's state changes.



## Why MADDPG
Traditional reinforcement learning methods are challenging to apply in multi-agent environments, primarily due to the fact that each agent's policy continuously changes during training. This leads to an unstable environment for each individual agent, where P(s'|s,a,π1,...,πN)≠P(s'|s,a,π1',...,πN') for any πi≠π'i. To a certain extent, optimizing policies for an agent based on this unstable environment becomes meaningless, as a policy optimized for the current state might become ineffective in the next changing environment state. This issue hinders the direct use of experience replay methods for training and is also the reason Q-learning fails. Additionally, for policy gradient methods, as the number of agents increases, the complexity of the environment increases as well. This leads to a significant increase in variance when estimating gradients through sampling.
These issues ultimately stem from a lack of interaction between agents. Agents are unaware of their teammates' or opponents' strategies, causing them to choose actions solely based on their own situations, thus ignoring the global context. Our solution to this problem is quite simple: we adopt a centralized training and decentralized execution approach. During training, we introduce a global-aware critic to guide the actor's training, while during testing, we use local-aware actors to take actions.
Multi-Agent Deep Deterministic Policy Gradient (MADDPG) is a deep reinforcement learning algorithm designed to solve cooperative decision-making problems in multi-agent environments. It extends the single-agent DDPG algorithm to be applicable in multi-agent scenarios.
The core idea of MADDPG is to treat each agent as an independent decision-making entity while considering the impact of other agents' actions on the current agent's decisions. This is achieved by utilizing two distinct networks: one for deterministic policy (actor) and the other for value function estimation (critic). Both of these networks are constructed using deep neural networks and optimized using gradient descent.
In the training process of MADDPG, each agent observes the environmental state and selects an action based on its own policy network. Essentially, MADDPG remains a DPG algorithm, training a centralized critic that requires global information for each agent, and a decentralized actor that requires only local observations. This structure allows MADDPG to be used in both cooperative and competitive scenarios. Since it is based on the DPG algorithm, it is compatible with continuous action spaces.
The training process of MADDPG involves multi-agent experience replay, similar to the single-agent version of DDPG. Each agent learns from its own experiences while leveraging information from other agents to improve the quality of decision-making. This approach proves effective in learning better strategies in multi-agent cooperation problems.


## Why M3DDPG
Agents trained by Deep Reinforcement Learning (DRL) often become fragile and are highly sensitive to the training environment, particularly in multi-agent scenarios. In multi-agent setups, the policies of DRL agents can easily get trapped in suboptimal local optima associated with their training partners. In other words, the learned policies might only be locally optimal with respect to the current strategies of other agents. To address this issue, we employ the M3DDPG algorithm.
"Minimax Deep Deterministic Policy Gradient" (M3DDPG) is a deep reinforcement learning algorithm designed to tackle multi-agent adversarial scenarios. It extends the DDPG algorithm and introduces the "Minimax" concept from game theory to effectively manage the competition and cooperation dynamics among multiple agents.
The core concept of M3DDPG treats the multi-agent adversarial scenario as a two-player zero-sum game, where the success of one agent implies the failure of another, and vice versa. In such a context, M3DDPG incorporates a "minimax" strategy optimization objective. This means that each agent aims to maximize its rewards while simultaneously minimizing the rewards of its opponents.
The training process of the M3DDPG algorithm revolves around two key aspects:
1. Minimization of Adversarial Reward: Each agent, while optimizing its strategy, considers the strategies of its opponents and the rewards they achieve. Pursuing the minimization of opponents' rewards enables the agent to better adapt to competitive environments.
2. Maximization of Self Reward: Similar to traditional reinforcement learning, each agent also strives to maximize its expected rewards to achieve its individual goals.
By simultaneously addressing these two aspects, the M3DDPG algorithm leverages an alternating training process to optimize the policy networks and value function networks of each agent. This optimization enables the agents to make strategic decisions in both competitive and cooperative scenarios effectively.


#  Studys of MARL application

### 1.Applications of Multi-Agent Reinforcement Learning in Robotic Collaboration- 
Robotic collaboration has emerged as a critical area of research, especially with the increasing complexity of tasks that robots are expected to perform. Multi-Agent Reinforcement Learning (MARL) provides a framework for multiple robots to learn and coordinate their actions in collaborative scenarios.- 
Task Allocation and Scheduling: In scenarios where multiple robots are working together, efficiently allocating tasks becomes crucial. MARL can be employed to develop algorithms that dynamically assign tasks to robots based on their capabilities and current states, ensuring optimal utilization of resources- .
Collaborative Manipulation: Tasks like lifting heavy objects or assembling parts often require multiple robots to work in tandem. MARL can be used to train robots to understand each other's actions and coordinate their movements for successful task completion.
Swarm Robotics: Swarm robotics involves the coordination of a large number of relatively simple robots to achieve a common goal. MARL provides a framework for these robots to learn from their interactions and adapt their behaviors to achieve tasks like exploration, mapping, and collective transpo- rt.
Communication and Information Sharing: In multi-robot systems, effective communication is vital. MARL can be employed to develop communication protocols where robots decide when and what information to share, ensuring efficient collaboration.
Adaptive Formation Control: In scenarios like search and rescue or surveillance, robots might need to maintain specific formations. MARL agents can be trained to adapt their positions dynamically based on the environment and task requirements.
In conclusion, MARL offers a robust approach to address the challenges posed by robotic collaboration. Its ability to learn from interactions and adapt in real-time makes it well-suited for scenarios where multiple robots need to work together seamlessly. As robotic systems become more prevalent in various sectors, the role of MARL in ensuring effective collaboration will undoubtedly grow.
E.g.
https://arxiv.org/pdf/2303.03381.pdf
https://www.mdpi.com/1424-8220/2### 3/7/3625

2.Applications of Multi-Agent Reinforcement Learning in Autonom- ous Driving
Autonomous driving is at the forefront of technological advancements in the transportation sector. As roads become more congested and diverse with a mix of autonomous and human-driven vehicles, the need for sophisticated decision-making algorithms becomes paramount. Multi-Agent Reinforcement Learning (MARL) offers a promising approach to address the challenges posed by the dynamic and interactive nature of traffic e- nvironments.
Intersection Management: Traditional traffic signal timings might not be optimal for intersections with a mix of autonomous and human-driven vehicles. MARL can be employed to develop dynamic traffic signal control strategies that adapt in real-time, ensuring smooth traffic flow and reducing w- aiting times.
Lane Change and Overtaking: Autonomous vehicles need to make safe and efficient lane change decisions, especially on highways. MARL agents can be trained to understand the behavior of surrounding vehicles and decide the best moments to change lanes or overtake.
Cooperative Driving: In scenarios where multiple autonomous vehicles are in proximity, they can cooperate to ensure optimal traffic flow. MARL can facilitate such cooperative behaviors, like platooning, where vehicles travel in closely spaced groups to reduce air drag-  and save fuel.
Avoidance of Traffic Congestions: Traffic congestions are not only time-consuming but also lead to increased fuel consumption. MARL agents can predict and avoid congested routes, ensuring faster and more efficient trips.
Emergency Maneuvers: In situations where quick reactions are required, like avoiding a pedestrian or a suddenly stopping vehicle, MARL can help autonomous vehicles take the best possible action to prevent accidents while considering the actions of other road users.
In conclusion, MARL provides a robust framework for developing decision-making algorithms for autonomous driving. Its ability to learn from interactions and adapt to changing environments makes it well-suited for the dynamic world of transportation. As autonomous driving technology continues to evolve, the role of MARL in ensuring safe and efficient vehicle operations will ### undoubtedly grow.

3.Applications of Multi-Agent Reinforcement Learning in S- mart Grid Management
The smart grid, an evolution of the traditional electrical grid, incorporates advanced technologies to enhance efficiency, reliability, and sustainability. Multi-Agent Reinforcement Learning (MARL) has shown potential in addressing the challenges of smart grid management, given the decentralized nature of the grid and the need for real-time decision-making.
Demand Response Management: One of the primary challenges in smart grid management is balancing supply and demand. MARL can be employed to develop dynamic pricing strategies, encouraging consumers to shift their consumption patterns, thereby ensuring grid stability.
Distributed Energy Resource (DER) Coordination: With the rise of renewable energy sources like solar and wind, coordinating these distributed resources becomes crucial. MARL agents can be trained to optimize the operation of DERs, maximizing energy production
- while minimizing costs.
Energy Storage Management: Energy storage systems, such as batteries, play a pivotal role in ensuring a continuous power supply. MARL can be used to develop strategies for optimal charging and discharging of these storage systems, considering factors like energy pri- ces and demand patterns.
Grid Security and Anomaly Detection: The smart grid is susceptible to various threats, including cyber-attacks. MARL agents can be trained to detect anomalies in the grid operation, ensuring timely intervention and minimizing potential damages.
Optimal Power Flow: Ensuring that electricity flows efficiently through the grid is essential for its reliable operation. MARL can be employed to optimize the power flow, considering constraints like transmission line capacities and voltage levels.
In conclusion, MARL offers promising solutions to the challenges of smart grid management. Its ability to learn and adapt in real-time makes it well-suited for the dynamic environment of the smart grid. As the grid continues to evolve, incorporating more renewable sources and advanced technologies, the role of MARL in ensuring its efficient and reliable op### eration will be paramount.

4.Applications of Multi-Agent Reinforcement L- earning in Online Ad Auctions
Online advertising has become an integral part of the- digital economy. With the rise of programmatic advertising and real-time bidding (RTB), the landscape of online ad auctions has evolved rapidly. Multi-Agent Reinforcement Learning (MARL) has emerged as a promising approach to address the challenges posed by the dynamic and competitive nature of these auctions.
Dynamic Bidding Strategies: Traditional bidding strategies in online ad auctions are often static and based on historical data. With MARL, agents can learn and adapt their bidding strategies in real-time, responding to the ever-changing auction environment and maximizing t- he return on advertising spend.
User Interaction Modeling: In the realm of online advertising, understanding user behavior is crucial. MARL can be employed to model the interactions between users and ads, helping advertisers to better target their audience and optimize ad placements.
Budget Allocation: For advertisers with multiple campaigns running simultaneously, allocating budget efficiently becomes a challenge. MARL agents can be trained to distribute budget across campaigns dynamically, ensuring that each campaign receives adequate funding whil- e maximizing overall performance.
Competitive Analysis: In RTB, advertisers often compete against each other for ad placements. MARL can be used to model the behavior of competing bidders, allowing advertisers to anticipate their competitors' actions and adjust their strategies accordingly.
Fraud Detection: Ad fraud is a persistent issue in online advertising. MARL agents can be trained to detect and counteract fraudulent activities, ensuring that advertisers get genuine impressions and clicks.
In conclusion, the application of MARL in online ad auctions presents numerous opportunities for enhancing bidding strategies, understanding user behavior, and optimizing campaign performance. As the online advertising ecosystem continues to evolve, the role of MARL in shaping its future is undeniable.
E.g.https://dl.### acm.org/doi/10.1145/3269206.3272021


5.Applications of Multi-Agent Reinforcem- ent Learning in Financial Market Trading
In recent years, the application of Multi-Agent Reinforcement Learning (MARL) in financial market trading has garnered significant attention. The complex and dynamic nature of financial markets makes them an ideal domain for the deployment of MARL, given its ability to model and opt- imize interactions among multiple agents.
Optimal Multi-Order Execution: Fang et al. (2023) introduced a MARL framework for multi-order execution in quantitative finance. Their approach treats each trading order as an individual agent, with agents collaborating to maximize overall profits. A novel aspect of their work is the development of a learnable multi-round communication protocol, allowing agents to share intended actions and refine them iteratively. This approach aims to replicate human traders' strategies, emphasizing real-world app- licability and efficient order execution. 
Strategic Trading in Quantitative Markets: Zhang et al. (2023) integrated traditional financial trading strategies with MADDPG, a MARL algorithm, to investigate strategic trading. Their methods, named CPPI-MADDPG and TIPP-MADDPG, were tested on real-world financial data, demonstrating superior performance over conventional strategies. This work highlights the potential of MARL to enhance established trading st- rategies through data-driven optimization. 
Regulatory Impact on Stock Market Quality: Lussange and Gutkin (2023) utilized a MARL-based agent model to study the effects of regulatory changes on stock market quality. They explored the implications of smaller order book tick sizes, larger metaorders, and increased trading frequencies. Their findings suggest that while higher trading frequencies can benefit market quality, the other two trends might not be as advantageous. This research underscores the importance of understanding the systemic complexities of financial markets and the potential-  role of MARL in regulatory decision-making.
Modeling Sub-Rational Human Investors: Liu et al. (2022) addressed the challenge of modeling human sub-rationality in financial decisions. Using a multi-agent market simulation, they investigated the relationship between investor profits and their degree of sub-rationality, attributing sub-rational behavior to psychological biases and computational limitations. This work emphasizes the need to consider human behavior nuances when designing MARL-based trading strategies. 
